# PUMA Challenge - Training HoVer-Net

This notebook demonstrates how to train the HoVer-Net model for the Panoptic Segmentation of nUclei and tissue in advanced MelanomA (PUMA) challenge. 

## PUMA Challenge Overview

The PUMA challenge consists of two tracks, each with two tasks:

### Track 1 – Panoptic segmentation with three instance classes:
- **Task 1**: Semantic tissue segmentation of tumor, stroma, epithelium, blood vessel, and necrotic regions.
- **Task 2**: Nuclei detection for three classes; tumor, TILs (lymphocytes and plasma cells), and other cells.

### Track 2 – Panoptic segmentation with ten instance classes:
- **Task 1**: Semantic tissue segmentation (same as Track 1).
- **Task 2**: Nuclei detection for all ten classes: tumor, lymphocytes, plasma cells, histiocytes, melanophages, neutrophils, stromal cells, epithelium, endothelium, and apoptotic cells.

In this notebook, we'll focus on training HoVer-Net for both tracks.

## 1. Environment Setup

First, let's set up our environment and import necessary libraries:

In [17]:
import os
import glob
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys
import json
import concurrent.futures  # Added for multi-threading

# Add HoVer-Net directory to path
sys.path.append('./hover_net/')

# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

CUDA available: False


## 2. Data Exploration and Understanding

The PUMA dataset consists of:
- 155 primary and 155 metastatic melanoma regions of interest (ROI), scanned at 40x magnification (1024 x 1024 pixels)
- Context ROI of 5120 x 5120 pixels, centered around each ROI
- Annotations for tissue and nuclei

Let's explore the dataset structure:

In [18]:
# Define dataset paths
dataset_path = "../dataset"

# List available ROIs
nuclei_annotations_path = os.path.join(dataset_path, "01_training_dataset_geojson_nuclei")
tissue_annotations_path = os.path.join(dataset_path, "01_training_dataset_geojson_tissue")
roi_images_path = os.path.join(dataset_path, "01_training_dataset_tif_ROIs")
context_roi_path = os.path.join(dataset_path, "01_training_dataset_tif_context_ROIs")

# Count the number of files in each directory
print(f"Number of nuclei annotation files: {len(os.listdir(nuclei_annotations_path))}")
print(f"Number of tissue annotation files: {len(os.listdir(tissue_annotations_path))}")
print(f"Number of ROI image files: {len(os.listdir(roi_images_path))}")
print(f"Number of context ROI image files: {len(os.listdir(context_roi_path))}")

# Look at sample filenames
print("\nSample nuclei annotation files:")
for file in os.listdir(nuclei_annotations_path)[:5]:
    print(f"  - {file}")

Number of nuclei annotation files: 205
Number of tissue annotation files: 205
Number of ROI image files: 205
Number of context ROI image files: 205

Sample nuclei annotation files:
  - training_set_metastatic_roi_080_nuclei.geojson
  - training_set_metastatic_roi_049_nuclei.geojson
  - training_set_primary_roi_023_nuclei.geojson
  - training_set_metastatic_roi_029_nuclei.geojson
  - training_set_metastatic_roi_065_nuclei.geojson


## 3. Data Preprocessing

HoVer-Net requires the data in a specific format. For training, we need to:
1. Convert GeoJSON annotations to instance masks
2. Extract patches from original images
3. Format data as required by HoVer-Net: [RGB, inst, type] channels

Let's create functions to handle this conversion:

In [19]:
import json
import cv2
import numpy as np
from shapely.geometry import Polygon, Point
import tifffile
import multiprocessing  # For CPU-bound operations


def read_geojson(geojson_path):
    """Read a GeoJSON file containing annotations."""
    with open(geojson_path) as f:
        geojson = json.load(f)
    return geojson


def geojson_to_mask_optimized(geojson, img_shape, class_mapping=None):
    """Convert GeoJSON annotations to instance and type masks with optimizations."""
    instance_mask = np.zeros(img_shape[:2], dtype=np.int32)
    type_mask = np.zeros(img_shape[:2], dtype=np.int32)

    # Pre-allocate arrays for better performance
    all_polygons = []
    all_instance_ids = []
    all_type_ids = []
    instance_id = 1  # Start with ID 1 (0 is background)

    # First, collect all polygons and their IDs
    for feature in geojson["features"]:
        cell_type = (
            feature["properties"].get("classification", {}).get("name", "unknown")
        )
        type_id = class_mapping.get(cell_type, 0) if class_mapping else 1

        # Get polygon coordinates
        if feature["geometry"]["type"] == "Polygon":
            coords = feature["geometry"]["coordinates"][0]
            # Convert to integer coordinates for cv2
            coords = np.array(coords, dtype=np.int32)

            all_polygons.append(coords)
            all_instance_ids.append(instance_id)
            all_type_ids.append(type_id)

            instance_id += 1

    # Fill all polygons at once when possible
    if len(all_polygons) > 0:
        # Process in batches to avoid memory issues with very large datasets
        batch_size = 1000  # Adjust based on memory constraints
        for i in range(0, len(all_polygons), batch_size):
            batch_polygons = all_polygons[i : i + batch_size]
            batch_instance_ids = all_instance_ids[i : i + batch_size]
            batch_type_ids = all_type_ids[i : i + batch_size]

            for polygon, inst_id, type_id in zip(
                batch_polygons, batch_instance_ids, batch_type_ids
            ):
                cv2.fillPoly(instance_mask, [polygon], inst_id)
                cv2.fillPoly(type_mask, [polygon], type_id)

    return instance_mask, type_mask


def extract_patches_optimized(
    image,
    instance_mask,
    type_mask,
    patch_size=270,
    stride=80,
    min_nuclei_percentage=0.05,
):
    """Extract patches more efficiently with parallel processing."""
    patches = []
    coords = []

    h, w = image.shape[:2]

    # Generate all potential patch coordinates
    patch_coords = []
    for y in range(0, h - patch_size + 1, stride):
        for x in range(0, w - patch_size + 1, stride):
            patch_coords.append((y, x))

    def process_patch(coord):
        y, x = coord
        inst_patch = instance_mask[y : y + patch_size, x : x + patch_size].copy()

        # Skip patches with no or few nuclei
        if np.max(inst_patch) == 0:
            return None

        # Skip if less than min_nuclei_percentage of patch has nuclei
        nuclei_pixels = np.sum(inst_patch > 0)
        total_pixels = patch_size * patch_size
        if nuclei_pixels / total_pixels < min_nuclei_percentage:
            return None

        img_patch = image[y : y + patch_size, x : x + patch_size].copy()
        type_patch = type_mask[y : y + patch_size, x : x + patch_size].copy()

        # Relabel instance IDs to be consecutive
        unique_ids = np.unique(inst_patch)
        unique_ids = unique_ids[unique_ids > 0]  # Skip background

        # Fast relabeling
        if len(unique_ids) > 0:
            remap = np.zeros(np.max(inst_patch) + 1, dtype=np.int32)
            remap[unique_ids] = np.arange(1, len(unique_ids) + 1)
            inst_patch = remap[inst_patch]

        # Stack data efficiently
        patch_data = np.dstack(
            [img_patch, inst_patch[..., None], type_patch[..., None]]
        )
        return patch_data

    # Process patches in parallel
    for patch in patch_coords:
        result = process_patch(patch)
        if result is not None:
            patches.append(result)

    return patches


def process_single_roi_optimized(
    roi_file, output_dir, class_mapping, nuclei_annotations_path
):
    """Process a single ROI image with optimizations."""
    roi_filename = os.path.basename(roi_file)
    roi_id = roi_filename.split(".")[0]  # Extract ID without extension

    # Find corresponding annotation file
    nuclei_geojson_file = os.path.join(
        nuclei_annotations_path, f"{roi_id}_nuclei.geojson"
    )

    if not os.path.exists(nuclei_geojson_file):
        print(f"WARNING: No annotation found for {roi_id}, skipping...")
        return 0

    # Load image and annotations
    image = tifffile.imread(roi_file)
    geojson = read_geojson(nuclei_geojson_file)

    # Convert annotations to masks using optimized function
    instance_mask, type_mask = geojson_to_mask_optimized(
        geojson, image.shape, class_mapping
    )

    # Extract patches using optimized function
    patches = extract_patches_optimized(image, instance_mask, type_mask)

    # Save patches in parallel
    def save_patch(args):
        j, patch = args
        patch_filename = f"{roi_id}_patch_{j}.npz"
        filepath = os.path.join(output_dir, patch_filename)
        np.savez_compressed(filepath, patch)

    for i, patch in enumerate(patches):
        save_patch((i, patch))

    return len(patches)


def process_dataset_optimized(output_dir, class_mapping, track_name, max_workers=None):
    """Process the PUMA dataset with optimized multiprocessing."""
    os.makedirs(output_dir, exist_ok=True)

    # Automatically determine optimal number of workers
    if max_workers is None:
        max_workers = os.cpu_count()

    # Get list of ROI images
    roi_files = sorted(glob.glob(os.path.join(roi_images_path, "*.tif")))

    print(f"Starting multiprocessing with {max_workers} workers...")

    # Prepare arguments for multiprocessing
    process_args = [
        (roi_file, output_dir, class_mapping, nuclei_annotations_path)
        for roi_file in roi_files
    ]

    # Use multiprocessing instead of threading for CPU-bound tasks
    with multiprocessing.Pool(processes=max_workers) as pool:
        results = []
        for i, num_patches in enumerate(
            pool.starmap(process_single_roi_optimized, process_args)
        ):
            roi_id = os.path.basename(roi_files[i]).split(".")[0]
            print(
                f"Processed {i+1}/{len(roi_files)}: {roi_id} - {num_patches} patches extracted"
            )
            results.append(num_patches)

    total_patches = sum(results)
    print(f"Finished processing {track_name}. Total patches saved: {total_patches}")

### Define Nuclear Type Mapping

We need to define how cell types in the PUMA dataset map to type IDs:

In [20]:
# Track 1 (3 classes) mapping
track1_mapping = {
    'tumor': 1,
    'lymphocyte': 2,  # TIL
    'plasma': 2,      # TIL
    'histiocyte': 3,  # Other
    'melanophage': 3, # Other
    'neutrophil': 3,  # Other
    'stromal': 3,     # Other
    'epithelium': 3,  # Other
    'endothelium': 3, # Other
    'apoptotic': 3,   # Other
    'unknown': 0      # Background
}

# Track 2 (10 classes) mapping
track2_mapping = {
    'tumor': 1,
    'lymphocyte': 2,
    'plasma': 3,
    'histiocyte': 4,
    'melanophage': 5,
    'neutrophil': 6,
    'stromal': 7,
    'epithelium': 8,
    'endothelium': 9,
    'apoptotic': 10,
    'unknown': 0     # Background
}

# For tissue segmentation
tissue_mapping = {
    'tumor': 1,
    'stroma': 2,
    'epithelium': 3,
    'blood_vessel': 4,
    'necrosis': 5,
    'unknown': 0     # Background
}

### Process Data to HoVer-Net Format

Let's create a function to process the dataset and save the patches:

### Run Data Processing

Let's create patches for both Track 1 and Track 2. Note that this may take some time.

In [21]:
# Set output directories
track1_patches_dir = "./processed_data/track1_patches"
track2_patches_dir = "./processed_data/track2_patches"

# Process data for Track 1 with multi-threading
# Adjust max_workers based on your CPU cores (default: 8)
process_dataset_optimized(track1_patches_dir, track1_mapping, "track1")

# Process data for Track 2 - uncomment to run
process_dataset_optimized(track2_patches_dir, track2_mapping, "track2")

Starting multiprocessing with 20 workers...


KeyboardInterrupt: 

### Split Data into Training and Validation Sets

Next, we'll split our processed patches into training and validation sets:

In [ ]:
import random
import shutil

def split_train_val_optimized(input_dir, train_dir, val_dir, val_ratio=0.2):
    """Split patches into training and validation sets with optimized file operations."""
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    
    all_patches = glob.glob(os.path.join(input_dir, "*.npy"))
    
    # Group patches by ROI ID to ensure patches from same ROI stay in same split
    roi_groups = {}
    for patch_path in all_patches:
        patch_name = os.path.basename(patch_path)
        roi_id = patch_name.split('_patch_')[0]
        if roi_id not in roi_groups:
            roi_groups[roi_id] = []
        roi_groups[roi_id].append(patch_path)
    
    # Split ROIs into train and validation
    all_rois = list(roi_groups.keys())
    random.shuffle(all_rois)
    val_size = int(len(all_rois) * val_ratio)
    val_rois = all_rois[:val_size]
    train_rois = all_rois[val_size:]
    
    # Function to copy files in parallel
    def copy_files_for_roi(args):
        roi, dest_dir = args
        copied = 0
        for patch_path in roi_groups[roi]:
            patch_name = os.path.basename(patch_path)
            dest_path = os.path.join(dest_dir, patch_name)
            shutil.copy(patch_path, dest_path)
            copied += 1
        return copied
    
    # Copy files in parallel
    train_args = [(roi, train_dir) for roi in train_rois]
    val_args = [(roi, val_dir) for roi in val_rois]
    
    with multiprocessing.Pool(processes=os.cpu_count()) as pool:
        train_copied = sum(pool.map(copy_files_for_roi, train_args))
        val_copied = sum(pool.map(copy_files_for_roi, val_args))
    
    print(f"Training set: {train_copied} patches from {len(train_rois)} ROIs")
    print(f"Validation set: {val_copied} patches from {len(val_rois)} ROIs")

In [ ]:
# Set directories for train/val splits
track1_train_dir = "./processed_data/track1_train"
track1_val_dir = "./processed_data/track1_val"
track2_train_dir = "./processed_data/track2_train"
track2_val_dir = "./processed_data/track2_val"

# Uncomment to run the splits
split_train_val_optimized(track1_patches_dir, track1_train_dir, track1_val_dir)
# split_train_val_optimized(track2_patches_dir, track2_train_dir, track2_val_dir)

Training set: 0 patches from 0 ROIs
Validation set: 0 patches from 0 ROIs


## 4. Configure HoVer-Net for PUMA Dataset

Now we need to create custom configuration files for HoVer-Net that specify our PUMA dataset:

In [ ]:
# Create a custom PUMA dataset class for Track 1 (3 classes)
puma_track1_definition = """

import numpy as np

def get_puma_track1():
    return {
        'hash': 'puma_track1',
        'name': 'puma_track1',
        'import_prep_func': None,
        'input_path': './processed_data/track1_train/',
        'split_info': {
            'train': {
                'input': './processed_data/track1_train/*.npy',
            },
            'valid': {
                'input': './processed_data/track1_val/*.npy',
            },
        },
        'type_info': {
            0: ['background', [0, 0, 0]],
            1: ['tumor', [255, 0, 0]],
            2: ['TILs', [0, 255, 0]],
            3: ['other', [0, 0, 255]],
        },
    }

def get_puma_track2():
    return {
        'hash': 'puma_track2',
        'name': 'puma_track2',
        'import_prep_func': None,
        'input_path': './processed_data/track2_train/',
        'split_info': {
            'train': {
                'input': './processed_data/track2_train/*.npy',
            },
            'valid': {
                'input': './processed_data/track2_val/*.npy',
            },
        },
        'type_info': {
            0: ['background', [0, 0, 0]],
            1: ['tumor', [255, 0, 0]],
            2: ['lymphocyte', [0, 255, 0]],
            3: ['plasma', [0, 0, 255]],
            4: ['histiocyte', [255, 255, 0]],
            5: ['melanophage', [255, 0, 255]],
            6: ['neutrophil', [0, 255, 255]],
            7: ['stromal', [128, 0, 0]],
            8: ['epithelium', [0, 128, 0]],
            9: ['endothelium', [0, 0, 128]],
            10: ['apoptotic', [128, 128, 128]],
        },
    }
"""

# Save the custom dataset definitions
with open("./hover_net/custom_puma_dataset.py", "w") as f:
    f.write(puma_track1_definition)

### Modify HoVer-Net Dataset Configuration

We need to register our custom dataset in the main dataset.py file:

In [ ]:
# Function to update dataset.py to include our custom dataset
def update_dataset_file():
    dataset_path = "./hover_net/dataset.py"
    
    # Read the original file
    with open(dataset_path, "r") as f:
        content = f.read()
    
    # Check if our dataset is already registered
    if "from custom_puma_dataset import get_puma_track1, get_puma_track2" in content:
        print("PUMA datasets already registered in dataset.py")
        return
    
    # Add import at the beginning
    import_line = "from custom_puma_dataset import get_puma_track1, get_puma_track2\n"
    # Look for other imports and add after them
    import_block_end = content.find("def get_dataset")
    if import_block_end > 0:
        content = content[:import_block_end] + import_line + content[import_block_end:]
    
    # Find the dataset_info dictionary and add our datasets
    dataset_dict_start = content.find("dataset_info = {")
    if dataset_dict_start > 0:
        dataset_dict_end = content.find("}", dataset_dict_start)
        if dataset_dict_end > 0:
            new_entries = """
    "puma_track1": get_puma_track1(),
    "puma_track2": get_puma_track2(),"""
            content = content[:dataset_dict_end] + new_entries + content[dataset_dict_end:]
    
    # Write back to file
    with open(dataset_path, "w") as f:
        f.write(content)
    
    print("Updated dataset.py with PUMA dataset configurations")

# Update the dataset file
update_dataset_file()

Updated dataset.py with PUMA dataset configurations


### Create Custom Configuration Files

Now let's create a custom configuration file for PUMA training:

In [ ]:
def create_custom_config(track_id=1, nr_classes=4):
    """
    Create a custom config file for HoVer-Net training on PUMA dataset.
    track_id: 1 or 2 (corresponding to Track 1 or Track 2)
    nr_classes: number of classes including background (4 for track1, 11 for track2)
    """
    config_content = f"""
import importlib
import random
import cv2
import numpy as np
from dataset import get_dataset

class Config(object):
    def __init__(self):
        self.seed = 10
        self.logging = True
        self.debug = False
        
        model_name = "hovernet"
        model_mode = "original"  # original or fast
        
        # Number of nuclear types (including background)
        nr_type = {nr_classes}  # {nr_classes-1} classes + background
        
        # Whether to predict the nuclear type
        self.type_classification = True
        
        # Shape information
        aug_shape = [540, 540]  # patch shape used during augmentation
        act_shape = [270, 270]  # patch shape used as input to network
        out_shape = [80, 80]    # patch shape at output of network
        
        # Dataset name
        self.dataset_name = "puma_track{track_id}"
        
        # Log directory for checkpoints
        self.log_dir = "logs/puma_track{track_id}/"
        
        # Paths to training and validation patches
        self.train_dir_list = [
            "./processed_data/track{track_id}_train"
        ]
        self.valid_dir_list = [
            "./processed_data/track{track_id}_val"
        ]
        
        self.shape_info = {{
            "train": {{
                "input_shape": act_shape,
                "mask_shape": out_shape,
            }},
            "valid": {{
                "input_shape": act_shape,
                "mask_shape": out_shape,
            }},
        }}
        
        # Parse config to the running state and set up associated variables
        self.dataset = get_dataset(self.dataset_name)
        module = importlib.import_module(
            "models.%s.opt" % model_name
        )
        self.model_config = module.get_config(nr_type, model_mode)
    """
    
    config_path = f"./hover_net/puma_track{track_id}_config.py"
    with open(config_path, "w") as f:
        f.write(config_content)
    
    print(f"Created custom config file at {config_path}")

# Create config files for both tracks
create_custom_config(track_id=1, nr_classes=4)  # Track 1: background + 3 classes
# create_custom_config(track_id=2, nr_classes=11)  # Track 2: background + 10 classes

Created custom config file at ./hover_net/puma_track1_config.py


## 5. Training HoVer-Net

With our data prepared and configuration files created, we're now ready to train HoVer-Net for the PUMA challenge. We'll train separate models for Track 1 and Track 2.

In [ ]:
# Check if our processed data is available
def check_processed_data():
    track1_train_files = glob.glob("./processed_data/track1_train/*.npy")
    track1_val_files = glob.glob("./processed_data/track1_val/*.npy")
    track2_train_files = glob.glob("./processed_data/track2_train/*.npy")
    track2_val_files = glob.glob("./processed_data/track2_val/*.npy")
    
    print(f"Track 1 Training Files: {len(track1_train_files)}")
    print(f"Track 1 Validation Files: {len(track1_val_files)}")
    print(f"Track 2 Training Files: {len(track2_train_files)}")
    print(f"Track 2 Validation Files: {len(track2_val_files)}")
    
    # Check a few files to see if they're properly formatted
    if len(track1_train_files) > 0:
        sample = np.load(track1_train_files[0])
        print(f"\nSample Training File Shape: {sample.shape}")
        print(f"Channels: RGB + Instance Mask + Type Mask")
        print(f"  - RGB shape: {sample[..., :3].shape}")
        print(f"  - Instance mask shape: {sample[..., 3].shape}")
        print(f"  - Type mask shape: {sample[..., 4].shape}")
        
        # Print some statistics
        print(f"\nInstance mask unique values: {np.unique(sample[..., 3])}")
        print(f"Type mask unique values: {np.unique(sample[..., 4])}")

# Check if our processed data is ready
check_processed_data()

Track 1 Training Files: 0
Track 1 Validation Files: 0
Track 2 Training Files: 0
Track 2 Validation Files: 0


### Run Training

Now we'll set up the code to run HoVer-Net training using our custom configuration.

In [ ]:
import sys
import os
import subprocess

def train_hovernet(track_id=1, gpu_ids="0"):
    """Train HoVer-Net for a specific PUMA track."""
    # Set environment variable for GPUs
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_ids
    
    # Command to run the training script
    cmd = [
        "python", 
        f"./hover_net/run_train.py", 
        f"--gpu={gpu_ids}"
    ]
    
    # Make sure the config file is properly imported
    original_config_path = "./hover_net/config.py"
    custom_config_path = f"./hover_net/puma_track{track_id}_config.py"
    
    # Backup original config
    if os.path.exists(original_config_path):
        with open(original_config_path, "r") as f:
            original_config_content = f.read()
        
        # Create backup
        with open(original_config_path + ".backup", "w") as f:
            f.write(original_config_content)
    
    # Copy our custom config to the main config.py file
    with open(custom_config_path, "r") as f:
        custom_config_content = f.read()
    
    with open(original_config_path, "w") as f:
        f.write(custom_config_content)
    
    print(f"Starting HoVer-Net training for PUMA Track {track_id}...")
    print(f"Using GPUs: {gpu_ids}")
    print(f"\nCommand: {' '.join(cmd)}\n")
    
    try:
        # Run the training script
        # In a real execution, you would uncomment the following:
        result = subprocess.run(cmd, check=True)
        print(f"Training completed with exit code {result.returncode}")
        print("[Simulated] Training would start here. In a real execution, uncomment the subprocess.run call.")
    except subprocess.CalledProcessError as e:
        print(f"Training failed with error: {e}")
    finally:
        # Restore original config if backup exists
        if os.path.exists(original_config_path + ".backup"):
            with open(original_config_path + ".backup", "r") as f:
                backup_content = f.read()
            
            with open(original_config_path, "w") as f:
                f.write(backup_content)
            
            # Remove backup
            os.remove(original_config_path + ".backup")

# Train for Track 1 (uncomment to run)
train_hovernet(track_id=1, gpu_ids="0")

# Train for Track 2 (uncomment to run)
# train_hovernet(track_id=2, gpu_ids="0")

Starting HoVer-Net training for PUMA Track 1...
Using GPUs: 0

Command: python ./hover_net/run_train.py --gpu=0



/home/alix_anneraud/Git/INSA/Deep_learning_project/code/.venv/lib/python3.12/site-packages/docopt.py:165: SyntaxWarning: invalid escape sequence '\S'
  name = re.findall('(<\S*?>)', source)[0]
/home/alix_anneraud/Git/INSA/Deep_learning_project/code/.venv/lib/python3.12/site-packages/docopt.py:166: SyntaxWarning: invalid escape sequence '\['
  value = re.findall('\[default: (.*)\]', source, flags=re.I)
/home/alix_anneraud/Git/INSA/Deep_learning_project/code/.venv/lib/python3.12/site-packages/docopt.py:207: SyntaxWarning: invalid escape sequence '\['
  matched = re.findall('\[default: (.*)\]', description, flags=re.I)
/home/alix_anneraud/Git/INSA/Deep_learning_project/code/.venv/lib/python3.12/site-packages/docopt.py:456: SyntaxWarning: invalid escape sequence '\S'
  split = re.split('\n *(<\S+?>|-\S+?)', doc)[1:]
Traceback (most recent call last):
  File "/home/alix_anneraud/Git/INSA/Deep_learning_project/code/./hover_net/run_train.py", line 37, in <module>
    from dataloader.train_loa

Training failed with error: Command '['python', './hover_net/run_train.py', '--gpu=0']' returned non-zero exit status 1.


### Monitor Training Progress

After starting the training, we can monitor its progress through TensorBoard logs. HoVer-Net training saves logs in the specified log directory.

In [ ]:
# Load TensorBoard extension
%load_ext tensorboard

# Start TensorBoard (uncomment to run)
# %tensorboard --logdir=logs/

## 6. Inference with Trained Models

After training, we can use the trained models to make predictions on new images. HoVer-Net provides a script for inference.

In [ ]:
def run_inference(track_id=1, gpu_ids="0"):
    """Run inference using a trained HoVer-Net model."""
    # Path to the trained model checkpoint
    checkpoint_path = f"./logs/puma_track{track_id}/hovernet_epoch=50.tar"
    
    # Path to test images
    test_dir = f"./test_images"
    output_dir = f"./results/track{track_id}"
    
    # Number of classes (including background)
    nr_types = 4 if track_id == 1 else 11
    
    # Command to run inference
    cmd = [
        "python", 
        "./hover_net/run_infer.py",
        "tile",
        f"--gpu={gpu_ids}",
        f"--nr_types={nr_types}",
        f"--model_path={checkpoint_path}",
        "--model_mode=original",
        f"--input_dir={test_dir}",
        f"--output_dir={output_dir}",
        "--save_qupath"
    ]
    
    print(f"Running inference with PUMA Track {track_id} model...")
    print(f"Using GPUs: {gpu_ids}")
    print(f"\nCommand: {' '.join(cmd)}\n")
    
    try:
        # Run the inference script
        # In a real execution, you would uncomment the following:
        # result = subprocess.run(cmd, check=True)
        # print(f"Inference completed with exit code {result.returncode}")
        print("[Simulated] Inference would start here. In a real execution, uncomment the subprocess.run call.")
    except subprocess.CalledProcessError as e:
        print(f"Inference failed with error: {e}")

# Run inference with Track 1 model (uncomment to run)
# run_inference(track_id=1, gpu_ids="0")

# Run inference with Track 2 model (uncomment to run)
# run_inference(track_id=2, gpu_ids="0")

## 7. Evaluate Results

Finally, we can evaluate the performance of our trained models on the validation set using various metrics.

In [ ]:
def evaluate_model(track_id=1):
    """Evaluate the trained model using metrics from HoVer-Net."""
    # Path to the results directory
    results_dir = f"./results/track{track_id}"
    
    # Command to compute metrics
    cmd = [
        "python", 
        "./hover_net/compute_stats.py",
        f"--pred_dir={results_dir}",
        f"--true_dir=./processed_data/track{track_id}_val"
    ]
    
    print(f"Computing metrics for PUMA Track {track_id} model...")
    print(f"\nCommand: {' '.join(cmd)}\n")
    
    try:
        # Run the metrics computation script
        # In a real execution, you would uncomment the following:
        # result = subprocess.run(cmd, check=True)
        # print(f"Metrics computation completed with exit code {result.returncode}")
        print("[Simulated] Metrics computation would start here. In a real execution, uncomment the subprocess.run call.")
    except subprocess.CalledProcessError as e:
        print(f"Metrics computation failed with error: {e}")

# Evaluate Track 1 model (uncomment to run)
# evaluate_model(track_id=1)

# Evaluate Track 2 model (uncomment to run)
# evaluate_model(track_id=2)

## 8. Conclusion

In this notebook, we've demonstrated the process of training HoVer-Net models for the PUMA challenge tasks. The steps included:

1. Exploring and understanding the PUMA dataset
2. Converting GeoJSON annotations to the format required by HoVer-Net
3. Processing images and generating patches for training
4. Creating custom configurations for HoVer-Net
5. Training separate models for Track 1 (3 classes) and Track 2 (10 classes)
6. Running inference on new images
7. Evaluating model performance with metrics

HoVer-Net's architecture, with its dedicated branches for segmentation and classification, is well-suited for the PUMA challenge which requires both accurate nuclei segmentation and classification.

To improve results, consider the following:
- Try different data augmentation techniques
- Adjust learning rates and training epochs
- Experiment with different model configurations (original vs. fast mode)
- Explore ensemble methods by combining predictions from multiple models